In [1]:
# Some parameters
Full = 'N'  # 9 Pannels isn't ideal for presentations N option give 4 plots
Year = 'Tav.nc'

In [2]:
# Load in Modules
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import os
import csv
import sys
#import glob
from scipy.interpolate import interp1d
from scipy import interpolate
from numba import autojit
from pylab import *
from matplotlib import gridspec
sys.path.append('/noc/users/hb1g13/Python/python_functions/')
from useful import *
from Flat_channel import *
import SG as SG
from HB_Plot import nf, fmt
%matplotlib inline

if Full == 'N':
    tau = ['3', '300', '3000', 'Closed']
elif Full == 'Extremes':
    tau = ['3','Closed']
else:
    tau = ['3', '10', '30', '100', '300',
           '1000', '3000', '10000', 'Closed']
Figletter = ['a) ','b) ','c) ','d) ']

# Path root
x = '/noc/msm/scratch/students/hb1g13/Mobilis'


# Now Make file structure
check = 0
runs = []
for i in range(len(tau)):
    flist = x+'/'+str(tau[i])+'daynokpp/'+Year
    if not os.path.exists(flist):
        print ' WARNING: '+flist+' does not exist! (skipping this tau...)'
        check += 0
    else:
        check += 1
        runs.append(i)
Runs=np.array(runs)
# Set Parameters
alpha = 2.000000E-04
g = -9.81
rho_0 = 1000
C_p = 3985
gridfilepath = x+'/3daynokpp/'
file2 = netCDF4.Dataset(gridfilepath+'grid.nc', 'r')
Zp = file2.variables['Zp1'][:]
Z = file2.variables['Z'][:]
Y = file2.variables['Yp1'][:]
Yc = file2.variables['Y'][:]
Zmatrix = np.zeros((len(Z), len(Yc)))
Zmatrix[:, :] = 1
dx = Y[1]-Y[0]  # Find Resolution
dz = Zp[0:len(Zp)-1]-Zp[1:len(Zp)]
dzc = Z[0:len(Z)-1]-Z[1:len(Z)]
ny = len(Y)
nz = len(Z)
Zmatrix[:,:] = np.apply_along_axis(np.multiply, 0, Zmatrix, dz)

In [4]:
# Surface Heat Fluxes
Ly = 2000e3
Lx = 1000e3
nx = np.round(Lx/dx)
ny = np.round(Ly/dx)
Q_0 = 10
Q = np.zeros_like(Yc)
pi = math.pi
Q=np.zeros(np.shape(Q))
Q[0:int(5*ny/36)] = Q_0*(np.cos(Yc[0:int(5*ny/36)]*(pi/(Yc[int(10*ny/36)]))))
Q[int(5*ny/36):int(20*ny/36)] = -Q_0*(np.sin((Yc[int(5*ny/36):int(20*ny/36)]-Yc[int(5*ny/36)])*(pi/(Yc[int(15*ny/36)]))))
Q[int(20*ny/36):int(30*ny/36)] = Q_0*(np.sin(Yc[0:int(10*ny/36)]*(pi/(Yc[int(10*ny/36)]))))
Psipred = -(Q)/(rho_0*C_p)

In [ ]:
fig = plt.figure(figsize=(16.5, 16.5))
for i in range(len(Runs)):
    ax = fig.add_subplot(2, 2, i+1)
    
    # Psi_res(y,z)
    
    fname = x+'/'+str(tau[Runs[i]])+'daynokpp/PSI.nc'
    file2read = netCDF4.Dataset(fname, 'r')
    lvrho = file2read.variables["LaVH1TH"][:]
    th = file2read.variables["LaHs1TH"][:].mean(axis=3).mean(axis=0)
    Va = file2read.variables["LaVa1TH"][:].mean(axis=3).mean(axis=0)
    P = file2read.variables["LaPs1TH"][:].mean(axis=3).mean(axis=0)
    dx = Y[1]-Y[0]  # Find Resolution
    VT = np.mean(lvrho, axis=3)  # Zonal average
    VTfdz = np.cumsum(VT[:, ::-1, :], axis=1)  # sum up the water column
    VTfdz = VTfdz
    # Time average and put into Sv and put back in right order
    psi = np.mean(VTfdz[:, ::-1, :], axis=0)
       
    # Temp field
    
    Rho = np.genfromtxt(x+'/'+str(tau[Runs[i]])+'daynokpp/Temp', delimiter=',')
    nolayers = len(psi[:, 1])
    Rho = Rho[0:nolayers]  # The layers package bins a layer so adjust
    Rho_matrix = np.apply_along_axis(np.multiply,0,ones_like(psi),Rho)
    T = numba_regridz(numba_regridy(get_psi_iso_z(Rho_matrix, th,Z)))
    
    for kk in range(len(psi[:,1])):
        if np.mean(psi[kk,10:-50])>0:
            if psi[kk,320]<0.9*np.max(psi[kk,10:-50]):
                continue
            psi[kk,:]=np.max(psi[kk,10:-50])
        else:
            if psi[kk,300]>0.5*np.min(psi[kk,10:-50]):
                continue
            psi[kk,:]=np.min(psi[kk,10:-50])
    
    psi[psi>1.5] = 1.5

    
    psiz = numba_regridz(numba_regridy(get_psi_iso_z(psi, th,Z)))
    Vaz = numba_regridz(numba_regridy(get_psi_iso_z(Va, th,Z)))
    Paz = numba_regridz(numba_regridy(get_psi_iso_z(Pa, th,Z)))
    # Mean Stream Function:
    fname = x+'/'+str(tau[Runs[i]])+'daynokpp/TAV.nc'
    file2 = netCDF4.Dataset(fname, 'r')
    TTav = file2.variables['THETA'][:].mean(axis=0).mean(axis=2)
    VTav = file2.variables['VVEL'][:].mean(axis=0).mean(axis=2)
    WTav = file2.variables['WVEL'][:].mean(axis=0).mean(axis=2)
    npad =((0,1),(0,0))
    WTav = np.pad(WTav,pad_width=npad, mode='constant', constant_values=0)
    ax = fig.add_subplot(2, 2, i+1)
    Eddy1 = x+'/'+str(tau[Runs[i]])+'daynokpp/TAV_VT.nc'
    f2 = netCDF4.Dataset(Eddy1,'r')
    VTTav = f2.variables['VVELTH'][:].mean(axis=0).mean(axis=2)
    WTTav = f2.variables['WVELTH'][:].mean(axis=0).mean(axis=2)
    WTTav = np.pad(WTTav,pad_width=npad, mode='constant', constant_values=0)
    VpTp = numba_regridy(VTTav) - numba_regridy(VTav)*TTav
    WpTp = numba_regridz(WTTav) - numba_regridz(WTav)*TTav
    Ty = ddy_cgrid_centered(TTav,Yc)
    Tz = ddz_cgrid_centered(TTav,Z)
    VTy = numba_regridy(VTav)*Ty
    WTz = numba_regridz(WTav)*Tz
    WpTpz = ddz_cgrid_centered(WpTp,Z)
    VpTpy = ddy_cgrid_centered(VpTp,Yc)
    Ty = ddy_cgrid_centered(TTav,Yc)
    Tz = ddz_cgrid_centered(TTav,Z)
    VTy = numba_regridy(VTav)*Ty
    WTz = numba_regridz(WTav)*Tz
    WpTpz = ddz_cgrid_centered(WpTp,Z)
    VpTpy = ddy_cgrid_centered(VpTp,Yc)
    Diff = np.nansum((VTy+WTz+WpTpz+VpTpy)*Zmatrix,axis=0)-Psipred
    np.apply_along_axis(np.multiply, 1, ones_like(Ty), Ty[0,:])
    Sp_est = - (np.apply_along_axis(np.multiply, 0, ones_like(Ty),Tz[:,-2])/
                (np.apply_along_axis(np.multiply, 1, ones_like(Ty), Ty[0,:])))
        
    Tavlat = TTav
    V = file2.variables["VVEL"][:]
    Vtave = np.mean(V, axis=0)
    Vtave[Vtave == 0] = np.nan
    Vzone = np.mean(Vtave, axis=2)
    # Got rid of for loop here (much quicker!!)
    psi2 = np.apply_along_axis(np.multiply, 0, Vzone, dz)
    psi = np.cumsum(-psi2[::-1, :], axis=0)
    npad = ((0, 1), (0, 0))
    # Pad with zeros at bottom
    psi = np.pad(psi, pad_width=npad, mode='constant',
                 constant_values=0)
    psiz = np.pad(psiz, pad_width=npad, mode='constant',
                 constant_values=0)
    Psied = psiz - numba_regridz(numba_regridy(psi))
  
    # Slope from layers
    SP = ddy_cgrid_centered(depths,Yc)
    SPZ = get_psi_iso_z(SP,th,Z)
    D = (ddy_cgrid_centered(VpTp-WpTp/numba_regridy(-SPZ),Y)*Zmatrix)[0::,:]
    # D = (ddz_cgrid_centered(WpTp-VpTp*numba_regridy(-SPZ),Z)*Zmatrix)[0::,:]
    for jj in range(len(Yc)):
        if np.where(Tz[2:,jj]==np.max(Tz[2:,jj]))[0][0]==0 and i<2:
            D[2::,jj] = 0
        else:
            D[np.where(Tz[2:,jj]==np.max(Tz[2:,jj]))[0][0]::,jj] = 0
    Diff[-25::]=0
    D = -np.sum(D,axis=0)
    #D = -np.sum((ddz_cgrid_centered(WpTp+Psied*Ty,Z)*Zmatrix)[0::,:],axis=0)
    Q = Psipred
    ROC = psiz[0,:]*Ty[0,:]
    ROC[-15::] = 0
    ROC[0] = 0

  #  p = plt.plot(Yc/1000,SG.savitzky_golay(ROC*10**5,15,1),'k',linewidth=2)
  #  p = plt.plot(Yc/1000,SG.savitzky_golay(D*10**5,27,1),'b',linewidth=2)
  #  p = plt.plot(Yc/1000,Q*10**5,'r',linewidth=2)
  #  p = plt.plot(Yc/1000,SG.savitzky_golay((ROC-Q)*10**5,15,1),'b',linewidth=1)
 #   plt.ylim(-1,1)

    ax.set_title(str(Figletter[Runs[i]])+str(tau[Runs[i]])+'day', fontsize=30)
    if str(tau[Runs[i]]) == 'Closed':
        ax.set_title(str(Figletter[Runs[i]])+str(tau[Runs[i]]), fontsize=30)
    ax.set_xlabel('Distance (km)', fontsize=30)
    ax.set_ylabel(r'$^oC m^2 s^{-1} $', fontsize=30)
    ax.tick_params(axis='both', which='major', labelsize=30)
plt.tight_layout()
lgd=plt.legend([r'$\Psi_{res}|_{hml}T_y$','D','Q'],bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.,fontsize=20)